# Introduction to Data Science – Lecture 15: Classification 1 – Decision Trees
*COMP 5360 / MATH 4100, University of Utah, http://datasciencecourse.net/*



You should install these two libraries to draw decision trees:
* PyDotPlus: `conda install pydotplus`
* GraphViz: Go [here](https://www.graphviz.org/download/) to download the app. On Mac and with homebrew you can run `brew install graphviz`. On Windows and with conda you can run `conda install graphviz`. 

Recommended Reading: 
* G. James, D. Witten, T. Hastie, and R. Tibshirani, An Introduction to Statistical Learning, Ch. 8 [digitial version available here](https://www.statlearning.com/)
* [Visual Intro to Machine Learning](http://www.r2d3.us/visual-intro-to-machine-learning-part-1/)

This lecture is in part based on [this lab](https://github.com/cs109/2015lab7/blob/master/Lab7-Botany%20and%20Ensemble%20Methods.ipynb), [this blog](http://hamelg.blogspot.com/2015/11/python-for-data-analysis-part-29.html), and the [scikit learn documentation](http://scikit-learn.org/stable/modules/tree.html).

In [ ]:
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import tree
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)
plt.style.use('ggplot')

## Data

We'll consider [a dataset of Titanic passengers](https://www.kaggle.com/c/titanic/) and develop a model to predict whether a particular passenger will survive or not. This is what [the dataset](titanic.csv) looks like:


| PassengerId | *Survived* | Pclass | Name                                                                               | Sex    | Age  | SibSp | Parch | Ticket             | Fare     | Cabin           | Embarked |
|-------------|----------|--------|------------------------------------------------------------------------------------|--------|------|-------|-------|--------------------|----------|-----------------|----------|
| 1           | 0        | 3      | Braund, Mr. Owen Harris                                                            | male   | 22   | 1     | 0     | A/5 21171          | 7.25     |                 | S        |
| 2           | 1        | 1      | Cumings, Mrs. John Bradley (Florence Briggs Thayer)                                | female | 38   | 1     | 0     | PC 17599           | 71.2833  | C85             | C        |
| 3           | 1        | 3      | Heikkinen, Miss. Laina                                                             | female | 26   | 0     | 0     | STON/O2. 3101282   | 7.925    |                 | S        |
| 4           | 1        | 1      | Futrelle, Mrs. Jacques Heath (Lily May Peel)                                       | female | 35   | 1     | 0     | 113803             | 53.1     | C123            | S        |
| 5           | 0        | 3      | Allen, Mr. William Henry                                                           | male   | 35   | 0     | 0     | 373450             | 8.05     |                 | S        |


Here are the variable descriptions for the non-obvious variables:
 * **Survived:**        (0 = No; 1 = Yes)
 * **pclass:**          Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd)
 * **sibsp:**           Number of Siblings/Spouses Aboard
 * **parch:**           Number of Parents/Children Aboard
 * **ticket:**          Ticket Number
 * **fare:**            Passenger Fare
 * **cabin:**           Cabin
 * **embarked:**        Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)


We're trying to decide whether a passenger with particular attributes will survive the Titanic disaster, so the **survival** variable is our label, the other columns are the **features**. 

## Decision Trees

[Decision trees](https://en.wikipedia.org/wiki/Decision_tree_learning) are very intuitive classification and regression tools. 

Consider the following [decision tree from the New York Times](https://archive.nytimes.com/www.nytimes.com/imagepages/2008/04/16/us/20080416_OBAMA_GRAPHIC.html), 
which predicts whether a county voted for Obama or Clinton in the 2008 democratic primary. 

<img src="oc-tree.jpeg" width="500">




Here is another decision tree for the survival of passengers on the titanic.  The figures under the leaves show the probability of survival and the percentage of observations in the leaf.

![](titanic_tree.png)

The use of a decision tree is very simple. Suppose someone gives you this tree and a new person. In order to predict whether or not the person would have died on the titanic, you ask the following questions, in order:

 * Is the person male? If no, we predict they would have survived. If yes, continue.
 * Is the person older than 9.5 years? If yes, we predict they would have died. If no, continue.
 * Did the person have 3 or more siblings? If yes, we predict they would have died. If no, they would have survived.
 
Many of these decisions make intuitive sense: social convention gave women and children preference for rescue. Some, however, are not as obvious: why is having more than 3 siblings or spouses a marker? Could this be a marker for a hidden variable?

The question we'll move to now is: How would one create such a tree? 

## Building a Decision Tree

Building a decision tree isn't really much harder than reading one. Here's the essential rundown of the [ID3 algorithm](https://en.wikipedia.org/wiki/ID3_algorithm):

 * If the data all have the same label, create a leaf node that predicts that label, and you're done.
 * If the list of attributes is empty (e.g., because you have used all already), create a leaf node that predicts the most common label.
 * Else, partition the data by each of the attributes; choose the attribute/partition with the lowest error.
 * Recursively continue on each partitioned subset using the remaining attributes.
 * Terminate when no attributes are left (see above), or when desired depth or another termination criterion is reached.
 

There is a very nice, [interactive tutorial on decision trees](http://www.r2d3.us/visual-intro-to-machine-learning-part-1/) that considers the problem of predicting whether a home is in *San Francisco* or *New York* based on attributes.
![](http://www.r2d3.us/static/pages/decision-trees-part-1/preview-en.png)

So, how do you choose the partition with the lowest error? There are various approaches.

Let's say we're building a classification tree by considering a list of predictors. In our example above we want to be able to classify whether people have survived based on things like gender, age, the booked fare, etc. Let's call the variables $X_{i,p}$ ($i$ for passengers, $p$ for predictors). Initially, for the first split, we consider all the passengers and all the predictors. We also have an observed label $Y_i$ for each passenger, and a predicted label $\hat{Y}_i$. 

We can calculate the *mean error*, 
$$
ME = \frac{1}{N}\sum_{i=1}^N \ell(\hat{Y}_i, Y_i),
$$ 
where $\ell(\hat{Y}_i, Y_i)$ is the error for sample $i$. Here, the error would be 
* squared error for regression, *i.e.*, $\ell(\hat{Y}_i, Y_i) = (\hat{Y}_i - Y_i)^2$ and 
* either [*cross-entropy*](https://en.wikipedia.org/wiki/Cross_entropy) 
or 
[*Gini impurity*](https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity) 
for classification. 


We want to achieve two things: pick the **best split** for the **best predictor**.

- At **each step** of the algorithm we consider a list of possible decisions or splits, *e.g.*, $X_{i,6} > 9$ (age is greater than 9), or $X_{i,5} = female$.
- For each possible decision we recalculate the predictor for that rule, for example $\hat{Y}_i = 1$ if $X_{i,6} > 9$ and $0$ otherwise.
- We recalculate the mean error for each possible decision
- We choose the decision that reduces the error by the largest amount.
- Then continue with the next step on the reduced input set.


In building decision trees, it is easy to overfit the data. There are several methods for avoiding this, which we'll discuss more below. Simple strategies include limiting the depth of a tree or only splitting when we have more than $N$ samples left. 

## Decision Trees with SciKit Learn

Scikit-learn has [a nice decision tree implementation](http://scikit-learn.org/stable/modules/tree.html) which we'll use to learn a tree for our Titanic dataset.

In [ ]:
titanic = pd.read_csv("titanic.csv")
titanic.head(15)

## Data cleanup

We need to do some cleanup: 
 * Age has missing values. Let's assume that a person with missing age is of mean age (this is not necessarily a good decision).
 * Embarked has missing values, we add a dedicated category for unknown embarkation points.
 * We need to convert the categorical values to numerical values. 

In [ ]:
titanic["Age"] = titanic["Age"].fillna(titanic["Age"].mean())

def sex_to_numeric(x):
    if x=='male':
        return 0
    if x=='female':
        return 1
    else:
        return x

titanic["Sex"] = titanic["Sex"].apply(sex_to_numeric)
titanic.head()

In [ ]:
# this will break if run more than once
def embarked_to_numeric(x):
    if x=="S":
        return 0
    if x=="C":
        return 1
    if x=="Q":
        return 2
    else: 
        return 3
    
titanic["Embarked"] = titanic["Embarked"].apply(embarked_to_numeric)
titanic.head(10)

Let's work only with the numerical and categorical variables and omit passengerID, Name, Ticket and Cabin. These values could contain some information, but it's hard to make sense of them without more context.

In [ ]:
features = ["Survived", "Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
titanic = titanic[features]
titanic.head(10)

## Exploring the data

Let's take a look at the data:

In [ ]:
titanic.describe()

**Questions:**
1. What percentage of passengers survived? 
2. What percentage of passengers are female?

In [ ]:
import seaborn as sns
sns.set()
sns.pairplot(titanic, hue="Survived");

It's difficult to find clear patterns here, except for the separation of Sex.

Here is a better plot, with a visualization technique called [parallel sets](http://bl.ocks.org/igorzilla/3086651):

![Parallel Sets](p_sets.png)


## Our first tree
Here is some code that splits the data into training and test sets for cross-validation and selects features.

In [ ]:
#labels =["Survived", "Perished"]

def splitData(features):
    """Split a subset of the titanic dataset, given by the features, into train and test sets."""
    titanic_predictors = titanic[features].values
    titanic_labels = titanic["Survived"].values

    # Split into training and test sets
    XTrain, XTest, yTrain, yTest = train_test_split(titanic_predictors, titanic_labels, random_state=1, test_size=0.5)
    return XTrain, XTest, yTrain, yTest

And more code for plotting decision trees:

In [ ]:
from IPython.display import Image, display  
import pydotplus 
from scipy import misc

def renderTree(my_tree, features):
    # hacky solution of writing to files and reading again
    # necessary due to library bugs
    filename = "temp.dot"
    with open(filename, 'w') as f:
        f = tree.export_graphviz(my_tree, 
                                 out_file=f, 
                                 feature_names=features, 
                                 class_names=["Perished", "Survived"],  
                                 filled=True, 
                                 rounded=True,
                                 special_characters=True)
  
    dot_data = ""
    with open(filename, 'r') as f:
        dot_data = f.read()

    graph = pydotplus.graph_from_dot_data(dot_data)
    image_name = "temp.png"
    graph.write_png(image_name)  
    display(Image(filename=image_name))

Now let's look at a decision tree that **ONLY operates on sex**! 

In [ ]:
decisionTree = tree.DecisionTreeClassifier()

XTrain, XTest, yTrain, yTest = splitData(["Sex"])
# fit the tree with the traing data
decisionTree = decisionTree.fit(XTrain, yTrain)

# predict with the training data
y_pred_train = decisionTree.predict(XTrain)
# measure accuracy
print('Accuracy on training data = ', metrics.accuracy_score(y_true = yTrain, y_pred = y_pred_train))

# predict with the test data
y_pred = decisionTree.predict(XTest)
# measure accuracy
print('Accuracy on test data = ', metrics.accuracy_score(y_true = yTest, y_pred = y_pred))

renderTree(decisionTree, ["Sex"])

~76% correct on the test set isn't bad! - sex seems to be a very good indicator of whether someone has survived or not.

## Adding more features

Let's add the number of siblings and spouses as a feature. 

In [ ]:
#  train tree on sex and the number of the number of siblings/spouses aboard
used_features = ["Sex", "SibSp"]
XTrain, XTest, yTrain, yTest = splitData(used_features)
decisionTree = tree.DecisionTreeClassifier()
decisionTree = decisionTree.fit(XTrain, yTrain)

y_pred_train = decisionTree.predict(XTrain)
print('Accuracy on training data= ', metrics.accuracy_score(y_true = yTrain, y_pred = y_pred_train))

y_pred = decisionTree.predict(XTest)
print('Accuracy on test data= ', metrics.accuracy_score(y_true = yTest, y_pred = y_pred))
renderTree(decisionTree, used_features)

Our accuracy on the training data has gone up, **but the accuracy on the test data has gone down**. It looks like we're overfitting. But maybe we just selected the wrong features? Let's just try all of them! 

In [ ]:
all_features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]

XTrain, XTest, yTrain, yTest = splitData(all_features)
decisionTree = tree.DecisionTreeClassifier()
decisionTree = decisionTree.fit(XTrain, yTrain)

y_pred_train = decisionTree.predict(XTrain)
print('Accuracy on training data= ', metrics.accuracy_score(y_true = yTrain, y_pred = y_pred_train))

y_pred = decisionTree.predict(XTest)
print('Accuracy on test data= ', metrics.accuracy_score(y_true = yTest, y_pred = y_pred))
renderTree(decisionTree, all_features)

OK, clearly, we're overfitting the data - 98% accuracy on the training data and only ~75% on the test data. Yet, we've created a complicated tree. 

Decision trees are notorious for overfitting the data. There are two parameters that help us reign in overfitting:

* **max_depth:** The maximum depth of the tree. If this is not set, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.
* **min_samples_split:** The minimum number of samples required to split an internal node: If the value is an integer, then consider `min_samples_split` as the minimum number. If it is float, then `min_samples_split` is a percentage and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split.

Let's try combinations of depth and min split size:

In [ ]:
# Limiting Depth!
decisionTree = tree.DecisionTreeClassifier(max_depth=3)

decisionTree = decisionTree.fit(XTrain, yTrain)

y_pred_train = decisionTree.predict(XTrain)
print('Accuracy on training data= ', metrics.accuracy_score(y_true = yTrain, y_pred = y_pred_train))

y_pred = decisionTree.predict(XTest)
print('Accuracy on test data= ', metrics.accuracy_score(y_true = yTest, y_pred = y_pred))
renderTree(decisionTree, all_features)

In [ ]:
# Lmiting the minimum samples used to split.
decisionTree = tree.DecisionTreeClassifier(min_samples_split=15)

decisionTree = decisionTree.fit(XTrain, yTrain)

y_pred_train = decisionTree.predict(XTrain)
print('Accuracy on training data= ', metrics.accuracy_score(y_true = yTrain, y_pred = y_pred_train))

y_pred = decisionTree.predict(XTest)
print('Accuracy on test data= ', metrics.accuracy_score(y_true = yTest, y_pred = y_pred))
renderTree(decisionTree, all_features)

In [ ]:
# Limiting Both

decisionTree = tree.DecisionTreeClassifier(max_depth=3, min_samples_split=10)
decisionTree = decisionTree.fit(XTrain, yTrain)

y_pred_train = decisionTree.predict(XTrain)
print('Accuracy on training data= ', metrics.accuracy_score(y_true = yTrain, y_pred = y_pred_train))

y_pred = decisionTree.predict(XTest)
print('Accuracy on test data= ', metrics.accuracy_score(y_true = yTest, y_pred = y_pred))
renderTree(decisionTree, all_features)

It looks like both, the minimum number of samples for splitting and the maximum depth help with overfitting and we achieve a 79-80% accuracy rate. That doesn't sound much better than just gender alone, but 4% improvement is a lot in classification.

Also, our last model is fairly simple yet quite accurate. The main point seems to be:

 * Sex is the dominant factor at the root of the tree
 * For females, if you're in class 1 or 2 you're almost sure to survive
 * For males, if you were younger than 6.5 years old, you had a chance to survive. 
 * Also note that there are branches that predict the same thing, but with different certainty. For example, in the male / adult category, if you were in "first class", you still were likely to die, but less likely than in second or third. 
 
Of course, here, we made these decisions on how to limit the tree manually. What would be a good approach to do this automatically? 

## Discussion 

### Advantages of decision trees
* Decision trees are simple to explain and interpret, and interpretability is a major issue in many applications. Think about credit decisions, or medical decisions. 
* There is a nice graphical display for trees
* Easy to handle categorical predictors, though with a caveat: SciKit Learn doesn't really support it (see "embarked" above). An alternative is to use [one-hot-encoding](https://en.wikipedia.org/wiki/One-hot); i.e., for each category you create a binary dimension "is in category or not". 

### Disadvantages of decision trees
* Decision trees generally don't have the predictive accuracy of other approaches as they tend to overfit the data. 
* Decision trees are non-robust, *i.e.*, sensitive to small changes in the data.

Both of these disadvantages are addressed by the following, more advanced methods. 

## Ensemble Methods based on Decision Trees

Ensemble Methods (Ensemble Learning) use multiple algorithms at the same time and then come to a consensus of a predictive label. You can read a little about such methods in ISL, Ch. 8.

One such method is **Bagging** (Bootstrap Aggregating). The idea of [bagging](https://en.wikipedia.org/wiki/Bootstrap_aggregating) is to generate several trained models (e.g., decision trees) based on subsets of the data and let the decision trees vote to arrive at a prediction.
Commonly the subset is chosen through bootstrapping, i.e., random sampling with replacement. 
Since averaging a set of observations reduces variances (think CLT), this increases the predictive accuracy of the method. 


[**Boosting**](https://en.wikipedia.org/wiki/Boosting_(machine_learning)) is similar to bagging, except that the trees are grown sequentially and are trained specifically to address previously mis-classified items.  

When applied to decision trees, these methods are called [Random Forests](https://en.wikipedia.org/wiki/Random_forest). Generally, random forests combine multiple decision trees that were generated with some randomness and let them vote on the result. Here the randomness comes from choosing a random sample of the prediction variables to build each tree. 


Here is an example with a random forest that uses *300* trees and bootstraping.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest =  RandomForestClassifier(bootstrap=True, n_estimators=300, random_state=0)

trained_forest = forest.fit(XTrain, yTrain)

y_pred_train = trained_forest.predict(XTrain)
print('Accuracy on training data= ', metrics.accuracy_score(y_true = yTrain, y_pred = y_pred_train))

y_pred = trained_forest.predict(XTest)
print('Accuracy on test data= ', metrics.accuracy_score(y_true = yTest, y_pred = y_pred))

Ensemble methods are generally of higher accuracy, but also take a lot longer to train. 